### 0. Imports and setup

In [1]:
from src.dataset import Colored3DMNIST, DataLoader
from src.plot import plot_img
from src.nn import UNet, ResNet_D
from pathlib import Path

import src.data as data
import numpy as np
import torchvision
import torch

In [2]:
if torch.cuda.is_available():
    torch.cuda.set_device(f"cuda:0")
torch.manual_seed(42)
np.random.seed(42)

### 1. Getting the data ready

In [3]:
if all([Path(f"data/{i}.npy").exists() for i in ["x_train_a", "x_train_b", "x_test"]]):
    X_RGB_train_A = np.load("data/x_train_a.npy")
    X_RGB_train_B = np.load("data/x_train_b.npy")
    X_RGB_test = np.load("data/x_test.npy")
else:
    X_RGB_train_A, X_RGB_train_B, X_RGB_test = data.get_dataset()

    data.save_data("data/x_train_a.npy", X_RGB_train_A)
    data.save_data("data/x_train_b.npy", X_RGB_train_B)
    data.save_data("data/x_test.npy", X_RGB_test)

In [4]:
train_a = Colored3DMNIST(X_RGB_train_A, transform=None)
train_a_loader = DataLoader(train_a, batch_size=32)

train_b = Colored3DMNIST(X_RGB_train_B, transform=None)
train_b_loader = DataLoader(train_b, batch_size=32)

test = Colored3DMNIST(X_RGB_test, transform=None)
test_loader = DataLoader(test, batch_size=32)

Let's plot a number:

In [6]:
def plot(img):
    if isinstance(img, torch.Tensor):
        img = img.detach().cpu().numpy()
    
    plot_img(img)

In [7]:
plot(next(iter(test_loader))[1])

### 3. Preparation for the training

In [ ]:
# Taken from https://github.com/iamalexkorotin/NeuralOptimalTransport/blob/main/src/tools.py
def weights_init_D(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='leaky_relu')
    elif classname.find('BatchNorm') != -1:
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)
        
def weights_init_mlp(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')

In [ ]:
f = ResNet_D(16, nc=3).cuda()
f.apply(weights_init_D)

T = UNet(3, 3, base_factor=48).cuda()

NameError: name 'ResNet_D' is not defined